In [ ]:
options(repos = c(CRAN = "https://packagemanager.posit.co/cran/__linux__/focal/latest"))
options()$repos
 
r_lib_path <- "/dbfs/R/library"
.libPaths(c(r_lib_path, .libPaths()))

pacman::p_load(
 sparklyr, 
 arrow,
 dplyr
)

In [ ]:
%fs ls /

In [ ]:
suppressWarnings(sc <- spark_connect(method = "databricks"))

#patient_admits <- spark_read_parquet(sc, "file:/dbfs/Data/CoreCDM/Silver/patient-admits/part-0.parquet")

patients <- spark_read_csv(sc, "file:/dbfs/Data/mimic-iv-demo/2.2/hosp/patients.csv.gz")
admissions <- spark_read_csv(sc, "file:/dbfs/Data/mimic-iv-demo/2.2/hosp/admissions.csv.gz")
labevents <- spark_read_csv(sc, "file:/dbfs/Data/mimic-iv-demo/2.2/hosp/labevents.csv.gz")
d_labitems <- spark_read_csv(sc, "file:/dbfs/Data/mimic-iv-demo/2.2/hosp/d_labitems.csv.gz")


In [ ]:
patients |>
head() |> 
display()


In [ ]:
%sql

#DBI::dbSendQuery(sc, "CREATE SCHEMA IF NOT EXISTS mimic_iv_demo;")

spark_write_table(admissions, "mimic_iv_demo.hosp_admissions", mode = "overwrite")
spark_write_table(patients, "mimic_iv_demo.hosp_patients", mode = "overwrite")
spark_write_table(labevents, "mimic_iv_demo.hosp_labevents", mode = "overwrite")
spark_write_table(labevents, "mimic_iv_demo.hosp_d_labitems", mode = "overwrite")


In [ ]:
%sql

select * from hive_metastore.mimic_iv_demo.hosp_${df_name}


In [ ]:
patients |>
spark_write_delta("file:/dbfs/Data/mimic-iv-demo/hosp/patients", 
options = c())